In [1]:
import pandas as pd 
import numpy as np 
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix, make_scorer
from sklearn.metrics import accuracy_score, f1_score, roc_curve, auc, roc_auc_score
from sklearn.model_selection import RepeatedKFold, KFold, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import xgboost
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.utils import class_weight
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.pipeline import Pipeline

In [2]:
# LEER LOS DATOS 
file_in = pd.read_csv('dead_due_to_AHE.csv') #lee archivo 
file_in = file_in.drop(['subject_id', 'hadm_id', 'icustay_id'], axis= 'columns') #elimino las columnas que no ocupo 

# IMPUTACIÓN DE DATOS 
x = file_in.drop(['dead'], axis='columns') #para mis variables de entrada soalamente elimino la varaible a predecir
y= file_in.dead #me quedo con la variable a predecir en y 
columnas= x.columns 


# DIVIDIR CONJUNTO DE ENTRENAMIENTO, VALIDACIÓN Y PRUEBA 
x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=0.3, )
print('X_train:', x_train.shape, 'X_test:', x_test.shape) 

#data = x.replace(" ", np.NaN)
#values = data.values
#imp = IterativeImputer(max_iter=10, random_state=0)
#imp = KNNImputer(n_neighbors=5, weights= 'uniform')
##imp = MissForest()

imp= SimpleImputer(missing_values=np.nan, strategy='constant').fit(x_train) #mean, most frequent, constant 
x_train = imp.fit_transform(x_train)
x_test = imp.fit_transform(x_test)
print('X_train:', x_train.shape, 'X_test:', x_test.shape)

X_train: (3563, 40) X_test: (1528, 40)
X_train: (3563, 40) X_test: (1528, 40)


In [3]:
print(type(x_train))
x_train_imputado = pd.DataFrame(x_train, columns=[columnas]) #convirtiendo el arreglo en dataframe 
x_test_imputado = pd.DataFrame(x_test, columns=[columnas])
x_train_imputado

<class 'numpy.ndarray'>


,age,kg,creatinine,sexo,first_sicu,first_micu,first_ccu,first_tsicu,first_csru,first_nicu,...,resprate_max,tempc_min,tempc_mean,tempc_max,spo2_min,spo2_mean,spo2_max,pao2_min,pao2_mean,pao2_max
0,90.0,88.599998,1.1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,36.0,35.833333,37.232540,39.555554,14.0,98.433225,100.0,105.0,184.800000,413.0
1,82.0,46.400002,1.5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,42.0,35.555556,36.856209,37.777778,88.0,96.160256,100.0,58.0,82.333333,133.0
2,75.0,83.400002,0.5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,28.0,34.055557,36.225343,38.666666,76.0,97.939655,100.0,55.0,99.111111,152.0
3,60.0,87.500000,0.9,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,18.0,37.166668,37.305556,37.444445,97.0,98.500000,100.0,0.0,0.000000,0.0
4,63.0,101.000000,3.6,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,44.0,35.722224,37.493651,40.000000,82.0,98.069106,100.0,53.0,143.029412,340.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3558,64.0,78.199997,1.4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,38.0,33.555556,36.328283,38.222224,84.0,97.149329,100.0,42.0,91.567742,266.0
3559,69.0,80.199997,0.8,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,24.0,36.277779,36.982456,37.500000,92.0,96.445783,100.0,0.0,0.000000,0.0
3560,51.0,87.699997,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,33.0,35.299999,36.545238,37.333332,83.0,99.590164,100.0,185.0,325.500000,485.0
3561,70.0,43.000000,1.2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,28.0,34.799999,37.408747,38.400002,90.0,95.933333,100.0,54.0,124.444444,192.0


In [4]:
# ESTANDARIZACIÓN DE DATOS 
estandarizar = ['age', 'kg', 'creatinine','heartrate_min', 'heartrate_mean', 'heartrate_max', 'sysbp_min',
       'sysbp_mean', 'sysbp_max', 'diasbp_min', 'diasbp_mean', 'diasbp_max',
       'meanbp_min', 'meanbp_mean', 'meanbp_max', 'resprate_min','resprate_mean', 'resprate_max', 'tempc_min', 'tempc_mean', 'tempc_max',
       'spo2_min', 'spo2_mean', 'spo2_max', 'pao2_min', 'pao2_mean','pao2_max']
dummies = ['sexo', 'first_sicu', 'first_micu',
       'first_ccu', 'first_tsicu', 'first_csru', 'first_nicu', 'last_sicu','last_micu', 'last_ccu', 'last_tsicu', 'last_csru', 'last_nicu']

#Intentar con las varaibles separadas y también juntas 
scaler = StandardScaler().fit(x_train_imputado)
#transformed_values[estandarizar] = scaler.fit_transform(transformed_values[estandarizar]) #estandarizo las variables númericas 
x_train_escalado = scaler.fit_transform(x_train_imputado)
x_test_escalado = scaler.fit_transform(x_test_imputado)

#COLOCAR PRINTS 
print('X_train:', x_train_escalado.shape, 'X_test:', x_test_escalado.shape)

X_train: (3563, 40) X_test: (1528, 40)


In [5]:
# DEFINIR MODELO A UTILIZAR 
#RF
modeloXGB = xgb.XGBClassifier(use_label_encoder=False)
modeloRF = RandomForestClassifier()
modeloSVM = svm.LinearSVC()
modeloADA = AdaBoostClassifier()
#DEFINIR PARÁMETROS A ITERAR

parametrosRF = {'n_estimators': range(10, 200, 10), 'criterion':['gini', 'entropy'], 'max_depth': range(1,10,1), 
              'max_features':['auto', 'sqrt', 'log2'],  'min_impurity_decrease':[1e-4, 1e-7], 
              'class_weight':['balanced', 'balanced_subsample']} # PARA RF

parametrosRFprueba = {'n_estimators': range(10, 200, 10),'max_depth': range(1,10,1),'class_weight':['balanced']}

ratio = float(np.sum(y_train == 0)) / np.sum(y_train==1)
                      
parametrosXGB = {'loss':['deviance', 'exponential'], 'criterion':['friedman_mse', 'mse', 'mae'], 'max_features':['auto', 'sqrt',
              'log2'], 'n_estimators': range(10, 200, 10), "learning_rate": [0.1,0.05],"colsample_bytree" : [0.4, 0.8],
                    "subsample" : [0.8, 0.4], "reg_alpha" : [0.5], "reg_lambda": [2],
                    "objective": ['binary:logistic'],  'max_depth': range(1,10,1), "gamma":[10],"rate_drop": [0.5, 0.3],
                    "seed": [422], "eval_metric": ['auc'],"scale_pos_weight": [ratio]}

parametrosXGBprueba = { 'n_estimators': range(10, 200, 10), "learning_rate": [0.1,0.05], 'max_depth': range(1,10,1),
                        "eval_metric": ['auc'],"scale_pos_weight": [ratio]}

parametrosSVM = {"C":np.logspace(-3,3,7), "class_weight": ['balanced'],"random_state": [422]}
parametrosSVMprueba = {'C':np.logspace(-3,3,7), 'loss':['hinge', 'squared_hinge'], 'multi_class':['ovr', 'crammer_singer']}

parametrosADA = {'n_estimators': range(10, 200, 10),  "learning_rate": [0.1,0.05], 'algorithm':['SAMME', 'SAMME.R']}

#ENTRENAR GRID SEARCH 
#TIPOS DE SCORE 
balanced = class_weight.compute_sample_weight('balanced', y_train)
RKF = RepeatedKFold(n_splits=5, n_repeats=2, random_state=422) #probar los CROSS VALIDATION
SKF = StratifiedKFold(n_splits=5, shuffle= True, random_state=422)
grid = GridSearchCV(estimator= modeloADA, param_grid= parametrosADA, cv=SKF, scoring='roc_auc', verbose=2, refit= True)

In [6]:
grid.fit(x_train, y_train, sample_weight = balanced)

Fitting 5 folds for each of 76 candidates, totalling 380 fits
[CV] END algorithm=SAMME, learning_rate=0.1, n_estimators=10; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.1, n_estimators=10; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.1, n_estimators=10; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.1, n_estimators=10; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.1, n_estimators=10; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.1, n_estimators=20; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.1, n_estimators=20; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.1, n_estimators=20; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.1, n_estimators=20; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.1, n_estimators=20; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.1, n_estimators=30; total time=   0.1s
[CV] END algorithm=SAMME, learning_rate=0.1, n_

[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=20; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=20; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=20; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=20; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=30; total time=   0.1s
[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=30; total time=   0.1s
[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=30; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=30; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=30; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=40; total time=   0.1s
[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=40; total time=   0.1s
[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=40; total time=   0.1s
[CV] END algorit

[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=30; total time=   0.1s
[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=30; total time=   0.1s
[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=30; total time=   0.1s
[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=40; total time=   0.1s
[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=40; total time=   0.1s
[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=40; total time=   0.1s
[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=40; total time=   0.1s
[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=40; total time=   0.1s
[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=50; total time=   0.2s
[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=50; total time=   0.2s
[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=50; total time=   0.2s
[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=50; total time=   0.2s
[CV]

[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=40; total time=   0.1s
[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=40; total time=   0.1s
[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=40; total time=   0.1s
[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=40; total time=   0.1s
[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=40; total time=   0.1s
[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=50; total time=   0.2s
[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=50; total time=   0.2s
[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=50; total time=   0.2s
[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=50; total time=   0.2s
[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=50; total time=   0.2s
[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=60; total time=   0.3s
[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=60; total time=

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=422, shuffle=True),
             estimator=AdaBoostClassifier(),
             param_grid={'algorithm': ['SAMME', 'SAMME.R'],
                         'learning_rate': [0.1, 0.05],
                         'n_estimators': range(10, 200, 10)},
             scoring='roc_auc', verbose=2)

In [7]:
# METRICAS DEL MODELO 
auc_train = grid.best_score_
std_train = grid.cv_results_['std_test_score'][grid.best_index_]
best_params = grid.best_params_
best_estimator = grid.best_estimator_
print("auc:",auc_train)
print("std_validation:",std_train)
print('best parameters:',best_params)
print('best estimator:', best_estimator)

auc: 0.8305901899048103
std_validation: 0.009966299877175984
best parameters: {'algorithm': 'SAMME.R', 'learning_rate': 0.1, 'n_estimators': 150}
best estimator: AdaBoostClassifier(learning_rate=0.1, n_estimators=150)
